In [43]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import re
import nltk
import time

from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
import json
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

In [4]:
df = pd.read_csv('resampled_review.csv')

In [7]:
df.value_counts('score')

score
1    8710
2    8710
3    8710
4    8710
5    8710
dtype: int64

In [14]:
label0 = df[df.score < 3]
label1 = df[df.score > 3]

In [66]:
test_set = df[df.score == 3]

In [15]:
label0 = label0.drop('score', axis=1)
label1 = label1.drop('score', axis=1)

In [16]:
label0['label'] = 0
label1['label'] = 1

In [20]:
train_set = pd.concat([label0, label1])

In [25]:
tokenizer = RegexpTokenizer(r'\w+')
train_set['content'] = train_set['content'].apply(lambda x: tokenizer.tokenize(x.lower()))

In [26]:
stopset = stopwords.words('english')

In [27]:
def remove_stopwords(text):
  words = [words for words in text if words not in stopset]
  return words

train_set['content'] = train_set['content'].apply(lambda x: remove_stopwords(x))

In [28]:
lemmatizer = WordNetLemmatizer()

In [33]:
def word_lemmatizer(text):
  lem_text = [lemmatizer.lemmatize(i) for i in text]
  return lem_text

train_set['content'] = train_set['content'].apply(lambda x: word_lemmatizer(x))

In [35]:
stemmer = PorterStemmer()

In [37]:
def word_stemmer(text):
  stem_text = " ".join([stemmer.stem(i) for i in text])
  return stem_text

train_set['content'] = train_set['content'].apply(lambda x: word_stemmer(x))

In [38]:
X_train, X_test, y_train, y_test = train_test_split(train_set['content'], train_set['label'], test_size = 0.33,
                                                    random_state = 78741)

In [41]:
cvec = CountVectorizer()
tfidf = TfidfVectorizer()

In [42]:
X_train_cvec = cvec.fit_transform(X_train)
X_test_cvec = cvec.transform(X_test)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [45]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 2.1 MB/s 
     |████████████████████████████████| 163 kB 58.9 MB/s 
     |████████████████████████████████| 6.6 MB 36.0 MB/s 


In [48]:
data = pd.concat([label0, label1])
sentences = data.content.values
labels = data.label.values

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(sentences)
vectorizer.vocabulary_

In [60]:
vectorizer.transform(sentences).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [61]:
from sklearn.model_selection import train_test_split

sentences_train, sentences_test, y_train, y_test = train_test_split(
   sentences, labels, test_size=0.25, random_state=1000)

In [62]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)
X_train

<26130x9876 sparse matrix of type '<class 'numpy.int64'>'
	with 351521 stored elements in Compressed Sparse Row format>

In [63]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.9591274397244547


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [73]:
X = test_set.content.values

In [76]:
X = vectorizer.transform(X).toarray()

In [80]:
y = classifier.predict(X)

In [81]:
test_set['label'] = y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
test_set

,content,score,label
17420,Selalu kalo mau pilih paket langsung loading m...,3,1
17421,Aplikasinya sering error.. ga bisa cek dan bel...,3,0
17422,The signal is getting worse. I can see the dif...,3,0
17423,Ga optimize untuk tab kah? Mau buy subscribtio...,3,0
17424,Di check lagi konfigurasi fortigate utk api se...,3,0
...,...,...,...
26125,"always ""something went wrong"" when i want to u...",3,1
26126,Sering banget eror jadi harus restart hp biar ...,3,0
26127,Where my internet connection? I'll working and...,3,0
26128,It's been 2 months since i use byU and just li...,3,1


In [83]:
score3 = test_set.drop('score', axis=1)

In [84]:
train_set

,content,label
0,permanenkan paket 1 5 mbp dan berikan opsi 30 ...,0
1,sinyal jelek padah kotaku telkomsel bagu,0
2,kenapa jaringannya sere lenyap sih padah bar s...,0
3,bad cheaper ye speed averag bought 2 mb packag...,0
4,keep crash last updat could cek data spend cou...,0
...,...,...
43545,work,1
43546,afford provid good signal,1
43547,help,1
43548,manjiw,1


In [85]:
data_review = pd.concat([train_set, score3])

In [86]:
data_review.to_csv('data_review.csv', index=None, header=True)